In [ ]:
%load_ext autoreload
%autoreload 2
import altair as alt
from alltime_athletics_viz.io import parse_running_events
import polars as pl
alt.data_transformers.disable_max_rows()


In [ ]:
df = parse_running_events()
df.groupby("event", maintain_order=True).first()

df = df.with_columns(  # add percentages w.r.t world records
    (
        100
        * (
            pl.col("result seconds")
            / pl.col("result seconds").head(10).last().over("event")
        )
    ).alias("percentage of 10th")
)

In [ ]:
df.head(3)

In [ ]:
legend_selection = alt.selection_point(fields=["event"], bind="legend")
chart = (
    alt.Chart(df.filter(pl.col("rank") < 1000))
    .mark_line(clip=True, point=True)
    # .mark_point(filled=True, size=10, clip=True)
    # .mark_line(clip=True, interpolate="step-after",point=True)
    .encode(
        # x=alt.X("rank:Q").scale(domain=(1, 51)),
        # x=alt.X("percentage of 10th:Q").scale(domain=(100, 103)),
        y=alt.Y("percentage of wr:Q").scale(domain=(100, 103)),
        # y=alt.Y("rank:Q").scale(domain=(10, 501)),
        x=alt.X("rank:Q").scale(domain=(1, 25)).axis(values=list(range(1, 26))),
        color=(
            alt.Color("event:N")
            .scale(zero=False, scheme="dark2")
            .legend(columns=1, symbolLimit=0, labelLimit=0)
        ),
        tooltip=["event:O"],
        # detail=alt.Detail(":N"),
        # column=alt.Column("distance type:N", spacing=50),
        # row="distance type:N",
        shape=alt.Shape("distance type:N"),
        detail="event:N",
        opacity=alt.condition(legend_selection, alt.value(1), alt.value(0.1)),
    )
    .properties(width=1400, height=500)
    # .resolve_scale(x="independent")
    # .resolve_scale(y="independent")
    .add_params(legend_selection)
    # .properties(title={"text": "TITLE GOES HERE", **Footer().subtitle()})
    # .configure_title(anchor="start")
    # .configure_axisX(titleAlign="right")
    # .configure_axisY(titleY=100, titleAlign="center", titleAngle=-90, titleX=-50)
    # .interactive()
)
chart

In [ ]:
df.groupby("event").first()